# Initialization

In [ ]:
%load_ext autoreload
%autoreload 2
import sys, warnings, time, numpy, yaml, pandas, logging
sys.path.append("../src/") # go to parent dir
from data_access import get_X, get_y, get_train_test
from models.factory import ModelFactory
warnings.filterwarnings('ignore')
with open('../confs/logs.yaml', 'rt') as f:
    config = yaml.safe_load(f.read())
logging.config.dictConfig(config)

In [ ]:
X_train, y_train, X_test, y_test, target = get_train_test(train_size=0.8, random_state=42)
train_scores = get_y()
test_data = get_X('test')
# features = ['TEAM_GAME_WON_season_sum', 'TEAM_GAME_WON_season_average', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_SHOTS_ON_TARGET_season_sum', 'TEAM_GAME_LOST_season_sum', 'TEAM_GAME_LOST_season_average', 'TEAM_ATTACKS_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_SHOTS_TOTAL_season_average', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_DANGEROUS_ATTACKS_season_sum', 'TEAM_GOALS_season_average', 'TEAM_YELLOWCARDS_5_last_match_std', 'TEAM_GOALS_season_sum', 'TEAM_CORNERS_season_average', 'TEAM_SHOTS_ON_TARGET_5_last_match_std', 'TEAM_YELLOWCARDS_season_std', 'TEAM_ATTACKS_5_last_match_std', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_SHOTS_OFF_TARGET_season_std']
# features = ['TEAM_GAME_WON_season_sum', 'TEAM_GOALS_season_average', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_GAME_LOST_season_sum', 'TEAM_GAME_WON_season_average', 'TEAM_GAME_LOST_season_average', 'TEAM_DANGEROUS_ATTACKS_season_sum', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_GOALS_season_sum', 'TEAM_CORNERS_season_sum', 'TEAM_SHOTS_TOTAL_season_average', 'TEAM_ATTACKS_season_average', 'TEAM_YELLOWCARDS_5_last_match_std', 'TEAM_ATTACKS_5_last_match_std', 'TEAM_SHOTS_ON_TARGET_season_sum', 'TEAM_SHOTS_ON_TARGET_5_last_match_std', 'TEAM_BALL_POSSESSION_season_std', 'TEAM_BALL_SAFE_season_std', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_BALL_SAFE_season_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_std', 'TEAM_CORNERS_season_average', 'TEAM_BALL_SAFE_5_last_match_average', 'TEAM_DANGEROUS_ATTACKS_season_std', 'TEAM_CORNERS_5_last_match_std', 'TEAM_ATTACKS_season_sum', 'TEAM_ATTACKS_season_std', 'TEAM_PENALTIES_season_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_std', 'TEAM_BALL_POSSESSION_5_last_match_std', 'TEAM_BALL_POSSESSION_5_last_match_average', 'TEAM_CORNERS_5_last_match_sum', 'TEAM_SAVES_5_last_match_std', 'TEAM_BALL_SAFE_5_last_match_std', 'TEAM_SHOTS_ON_TARGET_season_std', 'TEAM_SHOTS_OFF_TARGET_season_std', 'TEAM_GAME_DRAW_season_sum', 'TEAM_SAVES_season_std', 'TEAM_GOALS_5_last_match_std', 'TEAM_OFFSIDES_5_last_match_std', 'TEAM_OFFSIDES_season_std', 'TEAM_FOULS_season_std', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_std', 'TEAM_SUBSTITUTIONS_season_std', 'TEAM_YELLOWCARDS_season_std', 'TEAM_CORNERS_season_std', 'TEAM_SAVES_season_average', 'TEAM_CORNERS_5_last_match_average', 'TEAM_SAVES_season_sum', 'TEAM_FOULS_5_last_match_std', 'TEAM_YELLOWCARDS_5_last_match_sum', 'TEAM_BALL_SAFE_season_sum', 'TEAM_SUBSTITUTIONS_season_average', 'TEAM_REDCARDS_season_std', 'TEAM_YELLOWCARDS_season_sum', 'TEAM_OFFSIDES_season_sum', 'TEAM_SUBSTITUTIONS_5_last_match_std', 'TEAM_PENALTIES_season_average', 'TEAM_GAME_DRAW_season_average', 'TEAM_GOALS_season_std', 'TEAM_PENALTIES_season_sum', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_SHOTS_TOTAL_season_std', 'TEAM_GOALS_5_last_match_average', 'TEAM_SHOTS_INSIDEBOX_season_std', 'TEAM_SHOTS_TOTAL_5_last_match_std', 'TEAM_YELLOWCARDS_season_average', 'TEAM_SHOTS_INSIDEBOX_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_std', 'TEAM_SUBSTITUTIONS_season_sum', 'TEAM_OFFSIDES_5_last_match_sum', 'TEAM_OFFSIDES_season_average', 'TEAM_YELLOWCARDS_5_last_match_average', 'TEAM_OFFSIDES_5_last_match_average', 'TEAM_ATTACKS_5_last_match_average', 'TEAM_SHOTS_OUTSIDEBOX_season_std', 'TEAM_SHOTS_OFF_TARGET_season_sum', 'TEAM_SAVES_5_last_match_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_average', 'TEAM_SHOTS_ON_TARGET_5_last_match_average', 'TEAM_SHOTS_ON_TARGET_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_season_std', 'TEAM_REDCARDS_season_sum', 'TEAM_FOULS_season_average', 'TEAM_INJURIES_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std', 'TEAM_INJURIES_season_sum', 'TEAM_REDCARDS_season_average', 'TEAM_SHOTS_TOTAL_5_last_match_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_sum', 'TEAM_FOULS_season_sum', 'TEAM_DANGEROUS_ATTACKS_5_last_match_sum', 'TEAM_GOALS_5_last_match_sum', 'TEAM_SHOTS_OFF_TARGET_season_average', 'TEAM_SAVES_5_last_match_sum', 'TEAM_PASSES_season_std', 'TEAM_ATTACKS_5_last_match_sum', 'TEAM_SUBSTITUTIONS_5_last_match_average', 'TEAM_DANGEROUS_ATTACKS_5_last_match_average', 'TEAM_FOULS_5_last_match_sum', 'TEAM_GAME_LOST_5_last_match_average', 'TEAM_INJURIES_season_std', 'TEAM_SUBSTITUTIONS_5_last_match_sum', 'TEAM_PASSES_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average', 'TEAM_SHOTS_TOTAL_5_last_match_sum', 'TEAM_INJURIES_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_5_last_match_std', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_INJURIES_5_last_match_std', 'TEAM_FOULS_5_last_match_average', 'TEAM_INJURIES_season_average', 'TEAM_SHOTS_OUTSIDEBOX_season_average', 'TEAM_GAME_WON_5_last_match_average', 'TEAM_SHOTS_OUTSIDEBOX_season_sum', 'TEAM_GAME_WON_5_last_match_sum', 'TEAM_PASSES_season_average', 'TEAM_PENALTIES_5_last_match_std', 'TEAM_GAME_DRAW_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_season_average', 'TEAM_GAME_LOST_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_average', 'TEAM_SHOTS_INSIDEBOX_5_last_match_average', 'TEAM_PASSES_season_sum', 'TEAM_GAME_DRAW_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_season_sum', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_average', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_5_last_match_sum', 'TEAM_PENALTIES_5_last_match_average', 'TEAM_PENALTIES_5_last_match_sum', 'TEAM_REDCARDS_5_last_match_std', 'TEAM_PASSES_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_5_last_match_average', 'TEAM_PASSES_5_last_match_sum', 'TEAM_REDCARDS_5_last_match_average', 'TEAM_REDCARDS_5_last_match_sum']
# features=['TEAM_GAME_WON_season_average', 'TEAM_SHOTS_TOTAL_season_average', 'TEAM_SHOTS_ON_TARGET_5_last_match_std', 'TEAM_ATTACKS_5_last_match_std', 'TEAM_BALL_SAFE_season_std', 'TEAM_SHOTS_OFF_TARGET_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_std', 'TEAM_YELLOWCARDS_season_std', 'TEAM_SAVES_5_last_match_std', 'TEAM_SHOTS_OFF_TARGET_season_std', 'TEAM_ATTACKS_season_std', 'TEAM_OFFSIDES_season_std', 'TEAM_DANGEROUS_ATTACKS_season_std', 'TEAM_INJURIES_season_std', 'TEAM_BALL_POSSESSION_season_std', 'TEAM_YELLOWCARDS_5_last_match_std', 'TEAM_FOULS_5_last_match_std', 'TEAM_OFFSIDES_5_last_match_std', 'TEAM_BALL_SAFE_5_last_match_std', 'TEAM_BALL_POSSESSION_5_last_match_std', 'TEAM_SAVES_season_std', 'TEAM_SAVES_season_average', 'TEAM_BALL_SAFE_season_average', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_OFFSIDES_season_average', 'TEAM_SUBSTITUTIONS_season_average', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_GAME_LOST_season_sum', 'TEAM_FOULS_season_std', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_PASSES_5_last_match_std', 'TEAM_ATTACKS_season_average', 'TEAM_INJURIES_5_last_match_std', 'TEAM_INJURIES_season_sum', 'TEAM_PASSES_season_std', 'TEAM_BALL_SAFE_season_sum', 'TEAM_SUBSTITUTIONS_season_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_std', 'TEAM_GAME_LOST_season_average', 'TEAM_SHOTS_ON_TARGET_season_sum', 'TEAM_GOALS_season_sum', 'TEAM_CORNERS_5_last_match_std', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_std', 'TEAM_OFFSIDES_season_sum', 'TEAM_INJURIES_5_last_match_average', 'TEAM_GOALS_5_last_match_std', 'TEAM_SHOTS_TOTAL_season_std', 'TEAM_CORNERS_season_std', 'TEAM_SHOTS_OUTSIDEBOX_season_std', 'TEAM_CORNERS_season_sum', 'TEAM_GAME_DRAW_season_average', 'TEAM_CORNERS_season_average', 'TEAM_SHOTS_INSIDEBOX_5_last_match_std', 'TEAM_REDCARDS_season_std', 'TEAM_FOULS_5_last_match_sum', 'TEAM_GAME_DRAW_5_last_match_sum', 'TEAM_GAME_WON_5_last_match_sum', 'TEAM_SHOTS_TOTAL_5_last_match_std', 'TEAM_GAME_WON_season_sum', 'TEAM_YELLOWCARDS_5_last_match_sum', 'TEAM_SHOTS_OFF_TARGET_season_average', 'TEAM_SHOTS_ON_TARGET_season_std', 'TEAM_PENALTIES_5_last_match_std', 'TEAM_GAME_DRAW_season_sum', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_SUBSTITUTIONS_season_sum', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_GOALS_season_std', 'TEAM_PENALTIES_season_std', 'TEAM_FOULS_season_average', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std', 'TEAM_REDCARDS_season_average', 'TEAM_SAVES_5_last_match_sum', 'TEAM_ATTACKS_season_sum', 'TEAM_INJURIES_season_average', 'TEAM_FOULS_season_sum', 'TEAM_CORNERS_5_last_match_sum', 'TEAM_DANGEROUS_ATTACKS_season_sum', 'TEAM_YELLOWCARDS_season_sum', 'TEAM_SUCCESSFUL_PASSES_season_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_sum', 'TEAM_OFFSIDES_5_last_match_sum', 'TEAM_SHOTS_OUTSIDEBOX_season_average', 'TEAM_SHOTS_ON_TARGET_5_last_match_sum', 'TEAM_SUBSTITUTIONS_5_last_match_std', 'TEAM_PENALTIES_season_sum', 'TEAM_BALL_SAFE_5_last_match_average', 'TEAM_REDCARDS_5_last_match_std', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_sum', 'TEAM_INJURIES_5_last_match_sum', 'TEAM_SAVES_season_sum', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_average', 'TEAM_SHOTS_INSIDEBOX_season_std', 'TEAM_SUBSTITUTIONS_5_last_match_average', 'TEAM_PASSES_season_average', 'TEAM_SUCCESSFUL_PASSES_5_last_match_std', 'TEAM_REDCARDS_season_sum', 'TEAM_GOALS_5_last_match_average', 'TEAM_PASSES_season_sum', 'TEAM_GAME_LOST_5_last_match_sum', 'TEAM_SUBSTITUTIONS_5_last_match_sum', 'TEAM_BALL_POSSESSION_5_last_match_average', 'TEAM_GOALS_5_last_match_sum', 'TEAM_CORNERS_5_last_match_average', 'TEAM_GOALS_season_average', 'TEAM_ATTACKS_5_last_match_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_SHOTS_OFF_TARGET_season_sum', 'TEAM_PENALTIES_5_last_match_sum', 'TEAM_SHOTS_TOTAL_5_last_match_sum', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_5_last_match_sum', 'TEAM_SHOTS_TOTAL_5_last_match_average', 'TEAM_PASSES_5_last_match_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_average', 'TEAM_ATTACKS_5_last_match_average', 'TEAM_YELLOWCARDS_season_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_sum', 'TEAM_YELLOWCARDS_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_season_sum', 'TEAM_OFFSIDES_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_season_average', 'TEAM_SAVES_5_last_match_average', 'TEAM_DANGEROUS_ATTACKS_5_last_match_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_average', 'TEAM_PENALTIES_season_average', 'TEAM_PASSES_5_last_match_average', 'TEAM_FOULS_5_last_match_average', 'TEAM_SHOTS_OUTSIDEBOX_season_sum', 'TEAM_REDCARDS_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_5_last_match_average', 'TEAM_SHOTS_ON_TARGET_5_last_match_average', 'TEAM_PENALTIES_5_last_match_average', 'TEAM_GAME_WON_5_last_match_average', 'TEAM_REDCARDS_5_last_match_average', 'TEAM_GAME_LOST_5_last_match_average', 'TEAM_GAME_DRAW_5_last_match_average']
# features=features[:20]
# X_train = X_train[features]
# X_test = X_test[features]
# test_data = test_data[features]

In [ ]:
def eval_model(model, save_model=False, X=X_test):
    start = time.time()
    model.fit()
    score = model.evaluate(X)
    end = time.time()
    logging.info(f'{model.name}={score} in {numpy.round((end-start), 2)}s')
    if save_model:
        model.save(test_data)
    return {'name': model.name, 'score': score, 'time': numpy.round((end-start), 2)}

In [ ]:
%load_ext autoreload
%autoreload 2
def eval_model_for_name(name, X=X_train, y=y_train):
    with open('../confs/models.yaml', 'r') as file:
        configurations = yaml.safe_load(file)
    factory = ModelFactory(configurations, X, y, train_scores)
    model = factory.get_model(name)
    return eval_model(model)

In [ ]:
def get_features(name = 'random_forest'):
    with open('../confs/models.yaml', 'r') as file:
        configurations = yaml.safe_load(file)
    factory = ModelFactory(configurations, X_train, y_train, train_scores)
    model = factory.get_model(name)
    eval_model(model)
    df_importances = model.get_feature_importances()
    return list(df_importances['feature'])

# Feature importances

In [ ]:
names= ['random_forest', 'lightgbm', 'xgb_gblinear']
# for name in names:
#     logging.info(f'{name}: {get_features(name)}')
features = get_features(name = 'random_forest')
logging.info(features)
# features=['TEAM_GAME_WON_season_average', 'TEAM_GAME_LOST_season_average', 'TEAM_GOALS_season_sum', 'TEAM_GAME_LOST_season_sum', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_CORNERS_season_sum', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_BALL_POSSESSION_5_last_match_average', 'TEAM_ATTACKS_5_last_match_std', 'TEAM_YELLOWCARDS_5_last_match_std', 'TEAM_CORNERS_5_last_match_average', 'TEAM_BALL_POSSESSION_5_last_match_std', 'TEAM_SHOTS_ON_TARGET_5_last_match_std', 'TEAM_FOULS_season_std', 'TEAM_BALL_SAFE_season_average', 'TEAM_FOULS_5_last_match_std', 'TEAM_ATTACKS_season_sum', 'TEAM_BALL_POSSESSION_season_std', 'TEAM_SHOTS_ON_TARGET_season_std', 'TEAM_BALL_SAFE_season_sum', 'TEAM_YELLOWCARDS_season_std', 'TEAM_OFFSIDES_5_last_match_std', 'TEAM_PENALTIES_season_std', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_SUBSTITUTIONS_season_average', 'TEAM_GOALS_5_last_match_std', 'TEAM_SHOTS_OFF_TARGET_season_average', 'TEAM_YELLOWCARDS_season_sum', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_std', 'TEAM_REDCARDS_season_average', 'TEAM_GAME_DRAW_season_average', 'TEAM_DANGEROUS_ATTACKS_5_last_match_sum', 'TEAM_OFFSIDES_5_last_match_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_sum', 'TEAM_FOULS_season_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_average', 'TEAM_SAVES_5_last_match_average', 'TEAM_GOALS_5_last_match_average', 'TEAM_INJURIES_season_average', 'TEAM_FOULS_5_last_match_sum', 'TEAM_INJURIES_season_sum', 'TEAM_PASSES_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_season_std', 'TEAM_INJURIES_5_last_match_average', 'TEAM_REDCARDS_season_sum', 'TEAM_SUBSTITUTIONS_5_last_match_std', 'TEAM_INJURIES_season_std', 'TEAM_SUCCESSFUL_PASSES_5_last_match_std', 'TEAM_GAME_DRAW_5_last_match_sum', 'TEAM_SHOTS_TOTAL_5_last_match_average', 'TEAM_SHOTS_INSIDEBOX_5_last_match_average', 'TEAM_SHOTS_OUTSIDEBOX_season_sum', 'TEAM_GAME_LOST_5_last_match_sum', 'TEAM_PENALTIES_5_last_match_std', 'TEAM_SHOTS_TOTAL_5_last_match_sum', 'TEAM_PENALTIES_5_last_match_average', 'TEAM_REDCARDS_5_last_match_std', 'TEAM_PASSES_5_last_match_sum']
# features=['TEAM_GAME_WON_season_average', 'TEAM_GAME_LOST_season_average', 'TEAM_GOALS_season_sum', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_CORNERS_season_sum', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_YELLOWCARDS_5_last_match_std', 'TEAM_CORNERS_5_last_match_average', 'TEAM_BALL_POSSESSION_5_last_match_std', 'TEAM_SHOTS_ON_TARGET_5_last_match_std', 'TEAM_FOULS_season_std', 'TEAM_BALL_SAFE_season_average', 'TEAM_SHOTS_ON_TARGET_season_std', 'TEAM_YELLOWCARDS_season_std', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_SHOTS_OFF_TARGET_season_average', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_std', 'TEAM_GAME_DRAW_season_average', 'TEAM_DANGEROUS_ATTACKS_5_last_match_sum', 'TEAM_OFFSIDES_5_last_match_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_sum', 'TEAM_FOULS_season_average', 'TEAM_GOALS_5_last_match_average', 'TEAM_INJURIES_season_sum', 'TEAM_PASSES_5_last_match_std', 'TEAM_REDCARDS_season_sum', 'TEAM_SHOTS_OUTSIDEBOX_season_sum', 'TEAM_GAME_LOST_5_last_match_sum', 'TEAM_PENALTIES_5_last_match_average', 'TEAM_REDCARDS_5_last_match_std']
# features=['TEAM_GAME_WON_season_average', 'TEAM_GAME_LOST_season_average', 'TEAM_GOALS_season_sum', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_CORNERS_5_last_match_average', 'TEAM_FOULS_season_std', 'TEAM_BALL_SAFE_season_average', 'TEAM_SHOTS_ON_TARGET_season_std', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_GAME_DRAW_season_average', 'TEAM_DANGEROUS_ATTACKS_5_last_match_sum', 'TEAM_SHOTS_OFF_TARGET_5_last_match_sum', 'TEAM_FOULS_season_average', 'TEAM_PASSES_5_last_match_std', 'TEAM_REDCARDS_season_sum', 'TEAM_SHOTS_OUTSIDEBOX_season_sum']
# features=['TEAM_GAME_WON_season_average', 'TEAM_GAME_LOST_season_average', 'TEAM_GOALS_season_sum', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_CORNERS_5_last_match_average', 'TEAM_FOULS_season_std', 'TEAM_BALL_SAFE_season_average', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_GAME_DRAW_season_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_sum', 'TEAM_FOULS_season_average']
# features=['TEAM_GAME_WON_season_average', 'TEAM_GAME_LOST_season_average', 'TEAM_GOALS_season_sum', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_CORNERS_5_last_match_average', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_GAME_DRAW_season_average']

In [ ]:
def test_features(name = 'xgb_gblinear', save=True, features=None):
    with open('../confs/models.yaml', 'r') as file:
        configurations = yaml.safe_load(file)
    if features is None:
        features = get_features()
    lines = []
    previous_scores = 0
    selected_features = []
    for i in range(1, len(features)):
        sub_features = features[:i]
        factory = ModelFactory(configurations, X_train[sub_features], y_train, train_scores)
        model = factory.get_model(name)
        model.name = f'{name}_{i}'
        line = eval_model(model,X=X_test[sub_features])
        line['nbr_features'] = len(sub_features)
        line['model'] = name
        line['feature'] = sub_features[-1]
        line['features'] = sub_features
        lines.append(line)
        if previous_scores < line['score']:
            selected_features.append(line['feature'])
        previous_scores = line['score']
    df = pandas.DataFrame(lines)
    if save:
        df.to_csv(f'../data/output/features/{name}.csv')
    logging.info(f'Features selected: {len(selected_features)}, {selected_features}')
    return df

In [ ]:
name = 'lightgbm'
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_SHOTS_ON_TARGET_season_sum', 'TEAM_ATTACKS_season_sum', 'TEAM_ATTACKS_season_average', 'TEAM_GAME_LOST_season_sum', 'TEAM_ATTACKS_5_last_match_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_std', 'TEAM_CORNERS_5_last_match_std', 'TEAM_CORNERS_season_sum', 'TEAM_BALL_POSSESSION_5_last_match_average', 'TEAM_BALL_SAFE_season_std', 'TEAM_CORNERS_season_std', 'TEAM_FOULS_5_last_match_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_sum', 'TEAM_GOALS_5_last_match_std', 'TEAM_SAVES_5_last_match_std', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_PENALTIES_season_sum', 'TEAM_SHOTS_ON_TARGET_season_std', 'TEAM_BALL_SAFE_5_last_match_average', 'TEAM_PENALTIES_season_std', 'TEAM_SUBSTITUTIONS_season_average', 'TEAM_GAME_DRAW_season_sum', 'TEAM_YELLOWCARDS_season_sum', 'TEAM_SHOTS_OFF_TARGET_season_average', 'TEAM_SHOTS_TOTAL_season_std', 'TEAM_OFFSIDES_season_average', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_std', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_season_std', 'TEAM_FOULS_season_sum', 'TEAM_SUBSTITUTIONS_5_last_match_std', 'TEAM_SHOTS_OUTSIDEBOX_season_std', 'TEAM_ATTACKS_5_last_match_average', 'TEAM_REDCARDS_season_average', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_average', 'TEAM_INJURIES_season_std', 'TEAM_GOALS_5_last_match_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_sum', 'TEAM_INJURIES_season_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_average', 'TEAM_GAME_WON_5_last_match_average', 'TEAM_SUBSTITUTIONS_5_last_match_sum', 'TEAM_SUBSTITUTIONS_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_season_average', 'TEAM_GAME_DRAW_5_last_match_sum', 'TEAM_GAME_WON_5_last_match_sum', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_sum', 'TEAM_GAME_LOST_5_last_match_average', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_average', 'TEAM_PASSES_5_last_match_average', 'TEAM_PASSES_5_last_match_sum', 'TEAM_PASSES_season_sum', 'TEAM_SUCCESSFUL_PASSES_5_last_match_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_ATTACKS_season_sum', 'TEAM_GAME_LOST_season_sum', 'TEAM_CORNERS_season_sum', 'TEAM_BALL_SAFE_season_std', 'TEAM_CORNERS_season_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_sum', 'TEAM_SAVES_5_last_match_std', 'TEAM_PENALTIES_season_sum', 'TEAM_BALL_SAFE_5_last_match_average', 'TEAM_PENALTIES_season_std', 'TEAM_GAME_DRAW_season_sum', 'TEAM_SHOTS_OFF_TARGET_season_average', 'TEAM_OFFSIDES_season_average', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_std', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_std', 'TEAM_FOULS_season_sum', 'TEAM_SUBSTITUTIONS_5_last_match_std', 'TEAM_SHOTS_OUTSIDEBOX_season_std', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_average', 'TEAM_GOALS_5_last_match_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_season_average', 'TEAM_GAME_DRAW_5_last_match_sum', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_sum', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_average', 'TEAM_PASSES_5_last_match_sum', 'TEAM_PASSES_season_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_ATTACKS_season_sum', 'TEAM_CORNERS_season_sum', 'TEAM_BALL_SAFE_season_std', 'TEAM_CORNERS_season_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_sum', 'TEAM_PENALTIES_season_sum', 'TEAM_PENALTIES_season_std', 'TEAM_GAME_DRAW_season_sum', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_std', 'TEAM_FOULS_season_sum', 'TEAM_SUBSTITUTIONS_5_last_match_std', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_SHOTS_OFF_TARGET_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_5_last_match_std', 'TEAM_GAME_DRAW_5_last_match_sum', 'TEAM_PASSES_5_last_match_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_ATTACKS_season_sum', 'TEAM_BALL_SAFE_season_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_sum', 'TEAM_PENALTIES_season_std', 'TEAM_GAME_DRAW_season_sum', 'TEAM_FOULS_season_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_5_last_match_std']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_ATTACKS_season_sum', 'TEAM_BALL_SAFE_season_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_sum', 'TEAM_GAME_DRAW_season_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum']
df = test_features(name, features=features)
df[['score']].plot.line()

In [ ]:
name = 'ada_boost'
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_DANGEROUS_ATTACKS_season_sum', 'TEAM_SHOTS_TOTAL_season_average', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_GOALS_season_average', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_DANGEROUS_ATTACKS_season_std', 'TEAM_CORNERS_season_sum', 'TEAM_CORNERS_season_std', 'TEAM_SAVES_season_average', 'TEAM_BALL_SAFE_5_last_match_std', 'TEAM_GOALS_season_std', 'TEAM_FOULS_season_std', 'TEAM_SHOTS_OFF_TARGET_5_last_match_std', 'TEAM_BALL_POSSESSION_season_std', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_OFFSIDES_season_std', 'TEAM_BALL_SAFE_5_last_match_average', 'TEAM_SUBSTITUTIONS_season_average', 'TEAM_YELLOWCARDS_5_last_match_average', 'TEAM_OFFSIDES_season_average', 'TEAM_SAVES_5_last_match_average', 'TEAM_SUBSTITUTIONS_5_last_match_std', 'TEAM_SHOTS_OUTSIDEBOX_season_std', 'TEAM_YELLOWCARDS_5_last_match_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std', 'TEAM_SHOTS_OUTSIDEBOX_season_average', 'TEAM_PASSES_5_last_match_std', 'TEAM_SUBSTITUTIONS_5_last_match_average', 'TEAM_INJURIES_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_season_average', 'TEAM_PASSES_season_average', 'TEAM_PASSES_5_last_match_sum', 'TEAM_REDCARDS_5_last_match_std']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_DANGEROUS_ATTACKS_season_sum', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_CORNERS_season_sum', 'TEAM_FOULS_season_std', 'TEAM_BALL_POSSESSION_season_std', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_BALL_SAFE_5_last_match_average', 'TEAM_YELLOWCARDS_5_last_match_average', 'TEAM_SAVES_5_last_match_average', 'TEAM_SUBSTITUTIONS_5_last_match_std', 'TEAM_SHOTS_OUTSIDEBOX_season_std', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std', 'TEAM_PASSES_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_season_average']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_DANGEROUS_ATTACKS_season_sum', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_BALL_POSSESSION_season_std', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_BALL_SAFE_5_last_match_average', 'TEAM_SHOTS_OUTSIDEBOX_season_std', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_PASSES_5_last_match_std']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_DANGEROUS_ATTACKS_season_sum', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_DANGEROUS_ATTACKS_season_sum', 'TEAM_DANGEROUS_ATTACKS_season_average']
df = test_features(name, features=features)
df[['score']].plot.line()

In [ ]:
name = 'xgb_gblinear'
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_ATTACKS_season_sum', 'TEAM_GOALS_season_sum', 'TEAM_SHOTS_TOTAL_season_average', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_GAME_LOST_season_sum', 'TEAM_DANGEROUS_ATTACKS_season_std', 'TEAM_SHOTS_ON_TARGET_5_last_match_std', 'TEAM_SUBSTITUTIONS_season_std', 'TEAM_CORNERS_5_last_match_std', 'TEAM_BALL_POSSESSION_5_last_match_average', 'TEAM_BALL_SAFE_season_average', 'TEAM_BALL_SAFE_5_last_match_std', 'TEAM_GOALS_5_last_match_std', 'TEAM_SAVES_5_last_match_std', 'TEAM_SHOTS_OFF_TARGET_season_std', 'TEAM_CORNERS_5_last_match_sum', 'TEAM_YELLOWCARDS_season_average', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_OFFSIDES_season_std', 'TEAM_BALL_SAFE_5_last_match_average', 'TEAM_OFFSIDES_5_last_match_std', 'TEAM_YELLOWCARDS_season_sum', 'TEAM_OFFSIDES_5_last_match_sum', 'TEAM_SHOTS_TOTAL_season_std', 'TEAM_CORNERS_5_last_match_average', 'TEAM_REDCARDS_season_sum', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_season_std', 'TEAM_FOULS_season_sum', 'TEAM_SHOTS_OUTSIDEBOX_season_std', 'TEAM_YELLOWCARDS_5_last_match_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_average', 'TEAM_SHOTS_INSIDEBOX_season_std', 'TEAM_GOALS_5_last_match_sum', 'TEAM_SHOTS_TOTAL_5_last_match_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std', 'TEAM_GAME_WON_5_last_match_average', 'TEAM_INJURIES_5_last_match_sum', 'TEAM_SHOTS_OUTSIDEBOX_season_average', 'TEAM_SUBSTITUTIONS_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_5_last_match_std', 'TEAM_PENALTIES_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_season_average', 'TEAM_SUCCESSFUL_PASSES_season_sum', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average', 'TEAM_SHOTS_TOTAL_5_last_match_sum', 'TEAM_PENALTIES_5_last_match_sum', 'TEAM_PASSES_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_5_last_match_sum', 'TEAM_REDCARDS_5_last_match_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_ATTACKS_season_sum', 'TEAM_GOALS_season_sum', 'TEAM_SHOTS_TOTAL_season_average', 'TEAM_GAME_LOST_season_sum', 'TEAM_CORNERS_5_last_match_std', 'TEAM_BALL_SAFE_season_average', 'TEAM_SAVES_5_last_match_std', 'TEAM_CORNERS_5_last_match_sum', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_YELLOWCARDS_season_sum', 'TEAM_SHOTS_TOTAL_season_std', 'TEAM_FOULS_season_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_average', 'TEAM_GAME_WON_5_last_match_average', 'TEAM_INJURIES_5_last_match_sum', 'TEAM_SHOTS_OUTSIDEBOX_season_average', 'TEAM_PENALTIES_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_season_average', 'TEAM_SUCCESSFUL_PASSES_season_sum', 'TEAM_PASSES_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_5_last_match_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_ATTACKS_season_sum', 'TEAM_GOALS_season_sum', 'TEAM_SHOTS_TOTAL_season_average', 'TEAM_BALL_SAFE_season_average', 'TEAM_CORNERS_5_last_match_sum', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_SHOTS_TOTAL_season_std', 'TEAM_FOULS_season_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_GAME_WON_5_last_match_average', 'TEAM_SHOTS_OUTSIDEBOX_season_average', 'TEAM_PENALTIES_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_season_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_ATTACKS_season_sum', 'TEAM_GOALS_season_sum', 'TEAM_SHOTS_TOTAL_season_average', 'TEAM_BALL_SAFE_season_average', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_SHOTS_TOTAL_season_std', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_GAME_WON_5_last_match_average', 'TEAM_SHOTS_OUTSIDEBOX_season_average']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_ATTACKS_season_sum', 'TEAM_GOALS_season_sum', 'TEAM_SHOTS_TOTAL_season_average', 'TEAM_BALL_SAFE_season_average', 'TEAM_SHOTS_TOTAL_season_std', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum']
df = test_features(name, features=features)
df[['score']].plot.line()

In [ ]:
name = 'gradient_boosting'
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_GOALS_season_sum', 'TEAM_ATTACKS_season_average', 'TEAM_SHOTS_TOTAL_season_average', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_GAME_LOST_season_sum', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_DANGEROUS_ATTACKS_season_std', 'TEAM_YELLOWCARDS_5_last_match_std', 'TEAM_SUBSTITUTIONS_season_std', 'TEAM_CORNERS_5_last_match_std', 'TEAM_BALL_POSSESSION_5_last_match_std', 'TEAM_BALL_SAFE_season_average', 'TEAM_FOULS_5_last_match_std', 'TEAM_SAVES_season_average', 'TEAM_GOALS_season_std', 'TEAM_SHOTS_OFF_TARGET_season_std', 'TEAM_YELLOWCARDS_season_average', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_SAVES_season_sum', 'TEAM_PENALTIES_season_sum', 'TEAM_SHOTS_ON_TARGET_season_std', 'TEAM_OFFSIDES_season_std', 'TEAM_BALL_SAFE_5_last_match_average', 'TEAM_PENALTIES_season_std', 'TEAM_GAME_DRAW_season_sum', 'TEAM_GAME_DRAW_season_average', 'TEAM_YELLOWCARDS_5_last_match_average', 'TEAM_SHOTS_OFF_TARGET_season_average', 'TEAM_CORNERS_5_last_match_average', 'TEAM_REDCARDS_season_std', 'TEAM_OFFSIDES_season_average', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_std', 'TEAM_SHOTS_INSIDEBOX_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_season_std', 'TEAM_SAVES_5_last_match_average', 'TEAM_SHOTS_ON_TARGET_5_last_match_average', 'TEAM_SHOTS_OFF_TARGET_season_sum', 'TEAM_SHOTS_OUTSIDEBOX_season_std', 'TEAM_INJURIES_season_sum', 'TEAM_REDCARDS_season_average', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_OFFSIDES_5_last_match_average', 'TEAM_SHOTS_INSIDEBOX_5_last_match_average', 'TEAM_FOULS_5_last_match_average', 'TEAM_GOALS_5_last_match_sum', 'TEAM_GOALS_5_last_match_average', 'TEAM_SHOTS_TOTAL_5_last_match_average', 'TEAM_INJURIES_season_average', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std', 'TEAM_GAME_WON_5_last_match_average', 'TEAM_SHOTS_OUTSIDEBOX_season_average', 'TEAM_SUBSTITUTIONS_5_last_match_sum', 'TEAM_SUBSTITUTIONS_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_5_last_match_std', 'TEAM_SHOTS_OUTSIDEBOX_season_sum', 'TEAM_GAME_DRAW_5_last_match_sum', 'TEAM_INJURIES_5_last_match_average', 'TEAM_GAME_LOST_5_last_match_average', 'TEAM_SHOTS_TOTAL_5_last_match_sum', 'TEAM_PENALTIES_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_5_last_match_average', 'TEAM_PASSES_season_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_ATTACKS_season_average', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_BALL_POSSESSION_5_last_match_std', 'TEAM_FOULS_5_last_match_std', 'TEAM_SAVES_season_average', 'TEAM_SHOTS_OFF_TARGET_season_std', 'TEAM_YELLOWCARDS_season_average', 'TEAM_SAVES_season_sum', 'TEAM_PENALTIES_season_std', 'TEAM_YELLOWCARDS_5_last_match_average', 'TEAM_CORNERS_5_last_match_average', 'TEAM_REDCARDS_season_std', 'TEAM_SHOTS_INSIDEBOX_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_season_std', 'TEAM_SAVES_5_last_match_average', 'TEAM_SHOTS_ON_TARGET_5_last_match_average', 'TEAM_SHOTS_OFF_TARGET_season_sum', 'TEAM_INJURIES_season_sum', 'TEAM_REDCARDS_season_average', 'TEAM_OFFSIDES_5_last_match_average', 'TEAM_GOALS_5_last_match_sum', 'TEAM_SHOTS_TOTAL_5_last_match_average', 'TEAM_INJURIES_season_average', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std', 'TEAM_SHOTS_OUTSIDEBOX_season_average', 'TEAM_SHOTS_OUTSIDEBOX_season_sum', 'TEAM_GAME_LOST_5_last_match_average', 'TEAM_SHOTS_TOTAL_5_last_match_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_BALL_POSSESSION_5_last_match_std', 'TEAM_SHOTS_OFF_TARGET_season_std', 'TEAM_SAVES_season_sum', 'TEAM_PENALTIES_season_std', 'TEAM_YELLOWCARDS_5_last_match_average', 'TEAM_CORNERS_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_season_std', 'TEAM_SHOTS_ON_TARGET_5_last_match_average', 'TEAM_SHOTS_OFF_TARGET_season_sum', 'TEAM_REDCARDS_season_average', 'TEAM_GOALS_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std', 'TEAM_SHOTS_OUTSIDEBOX_season_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_SAVES_season_sum', 'TEAM_PENALTIES_season_std', 'TEAM_YELLOWCARDS_5_last_match_average', 'TEAM_SHOTS_ON_TARGET_5_last_match_average', 'TEAM_SHOTS_OFF_TARGET_season_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_SAVES_season_sum', 'TEAM_SHOTS_ON_TARGET_5_last_match_average']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_DANGEROUS_ATTACKS_season_sum', 'TEAM_DANGEROUS_ATTACKS_season_average']
df = test_features(name, features=features)
df[['score']].plot.line()

In [ ]:
name = 'ada_boost'
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_DANGEROUS_ATTACKS_season_sum', 'TEAM_SHOTS_TOTAL_season_average', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_GOALS_season_average', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_DANGEROUS_ATTACKS_season_std', 'TEAM_CORNERS_season_sum', 'TEAM_CORNERS_season_std', 'TEAM_SAVES_season_average', 'TEAM_BALL_SAFE_5_last_match_std', 'TEAM_GOALS_season_std', 'TEAM_FOULS_season_std', 'TEAM_SHOTS_OFF_TARGET_5_last_match_std', 'TEAM_BALL_POSSESSION_season_std', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_OFFSIDES_season_std', 'TEAM_BALL_SAFE_5_last_match_average', 'TEAM_SUBSTITUTIONS_season_average', 'TEAM_YELLOWCARDS_5_last_match_average', 'TEAM_OFFSIDES_season_average', 'TEAM_SAVES_5_last_match_average', 'TEAM_SUBSTITUTIONS_5_last_match_std', 'TEAM_SHOTS_OUTSIDEBOX_season_std', 'TEAM_YELLOWCARDS_5_last_match_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std', 'TEAM_SHOTS_OUTSIDEBOX_season_average', 'TEAM_PASSES_5_last_match_std', 'TEAM_SUBSTITUTIONS_5_last_match_average', 'TEAM_INJURIES_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_season_average', 'TEAM_PASSES_season_average', 'TEAM_PASSES_5_last_match_sum', 'TEAM_REDCARDS_5_last_match_std']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_DANGEROUS_ATTACKS_season_sum', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_CORNERS_season_sum', 'TEAM_FOULS_season_std', 'TEAM_BALL_POSSESSION_season_std', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_BALL_SAFE_5_last_match_average', 'TEAM_YELLOWCARDS_5_last_match_average', 'TEAM_SAVES_5_last_match_average', 'TEAM_SUBSTITUTIONS_5_last_match_std', 'TEAM_SHOTS_OUTSIDEBOX_season_std', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std', 'TEAM_PASSES_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_season_average']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_DANGEROUS_ATTACKS_season_sum', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_BALL_POSSESSION_season_std', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_BALL_SAFE_5_last_match_average', 'TEAM_SHOTS_OUTSIDEBOX_season_std', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_PASSES_5_last_match_std']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_DANGEROUS_ATTACKS_season_sum', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_GAME_WON_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_DANGEROUS_ATTACKS_season_sum', 'TEAM_DANGEROUS_ATTACKS_season_average']
df = test_features(name, features=features)
df[['score']].plot.line()

In [ ]:
name = 'hist_gradient_boosting'
features=['TEAM_GAME_WON_season_sum', 'TEAM_ATTACKS_season_sum', 'TEAM_DANGEROUS_ATTACKS_season_sum', 'TEAM_GOALS_season_sum', 'TEAM_SHOTS_TOTAL_season_average', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_GAME_LOST_season_sum', 'TEAM_DANGEROUS_ATTACKS_season_std', 'TEAM_CORNERS_5_last_match_std', 'TEAM_CORNERS_season_sum', 'TEAM_BALL_SAFE_season_std', 'TEAM_BALL_SAFE_season_average', 'TEAM_CORNERS_season_std', 'TEAM_SAVES_season_average', 'TEAM_SAVES_5_last_match_std', 'TEAM_SHOTS_OFF_TARGET_5_last_match_std', 'TEAM_CORNERS_5_last_match_sum', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_PENALTIES_season_sum', 'TEAM_BALL_SAFE_season_sum', 'TEAM_ATTACKS_season_std', 'TEAM_PENALTIES_season_std', 'TEAM_OFFSIDES_5_last_match_std', 'TEAM_GAME_DRAW_season_average', 'TEAM_SUBSTITUTIONS_season_sum', 'TEAM_YELLOWCARDS_season_sum', 'TEAM_OFFSIDES_5_last_match_sum', 'TEAM_SHOTS_TOTAL_season_std', 'TEAM_OFFSIDES_season_average', 'TEAM_SHOTS_INSIDEBOX_5_last_match_std', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_std', 'TEAM_PENALTIES_season_average', 'TEAM_SUBSTITUTIONS_5_last_match_std', 'TEAM_ATTACKS_5_last_match_sum', 'TEAM_SHOTS_OFF_TARGET_season_sum', 'TEAM_ATTACKS_5_last_match_average', 'TEAM_REDCARDS_season_average', 'TEAM_OFFSIDES_5_last_match_average', 'TEAM_SHOTS_INSIDEBOX_5_last_match_average', 'TEAM_GAME_LOST_5_last_match_sum', 'TEAM_INJURIES_season_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_average', 'TEAM_SHOTS_ON_TARGET_5_last_match_sum', 'TEAM_PASSES_season_std', 'TEAM_SHOTS_OFF_TARGET_5_last_match_average', 'TEAM_GAME_WON_5_last_match_average', 'TEAM_FOULS_season_average', 'TEAM_INJURIES_5_last_match_sum', 'TEAM_SUBSTITUTIONS_5_last_match_sum', 'TEAM_INJURIES_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_5_last_match_std', 'TEAM_PENALTIES_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_season_average', 'TEAM_INJURIES_5_last_match_average', 'TEAM_GAME_WON_5_last_match_sum', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average', 'TEAM_PASSES_season_average', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_5_last_match_average', 'TEAM_PENALTIES_5_last_match_average', 'TEAM_REDCARDS_5_last_match_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_GOALS_season_sum', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_GAME_LOST_season_sum', 'TEAM_CORNERS_5_last_match_std', 'TEAM_CORNERS_season_std', 'TEAM_SAVES_5_last_match_std', 'TEAM_CORNERS_5_last_match_sum', 'TEAM_BALL_SAFE_season_sum', 'TEAM_ATTACKS_season_std', 'TEAM_PENALTIES_season_std', 'TEAM_YELLOWCARDS_season_sum', 'TEAM_OFFSIDES_5_last_match_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_std', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_std', 'TEAM_SUBSTITUTIONS_5_last_match_std', 'TEAM_SHOTS_OFF_TARGET_season_sum', 'TEAM_ATTACKS_5_last_match_average', 'TEAM_OFFSIDES_5_last_match_average', 'TEAM_INJURIES_season_std', 'TEAM_SHOTS_ON_TARGET_5_last_match_sum', 'TEAM_PASSES_season_std', 'TEAM_FOULS_season_average', 'TEAM_INJURIES_5_last_match_sum', 'TEAM_SUBSTITUTIONS_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_season_average', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average', 'TEAM_PASSES_season_average', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_5_last_match_average']
features=['TEAM_GAME_WON_season_sum', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_SAVES_5_last_match_std', 'TEAM_BALL_SAFE_season_sum', 'TEAM_PENALTIES_season_std', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_std', 'TEAM_ATTACKS_5_last_match_average', 'TEAM_OFFSIDES_5_last_match_average', 'TEAM_SHOTS_ON_TARGET_5_last_match_sum', 'TEAM_FOULS_season_average', 'TEAM_SUBSTITUTIONS_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_season_average', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_average']
features=['TEAM_GAME_WON_season_sum', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_PENALTIES_season_std', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_std', 'TEAM_SHOTS_ON_TARGET_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_season_average']
features=['TEAM_GAME_WON_season_sum', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_SHOTS_ON_TARGET_5_last_match_sum']
df = test_features(name, features=features)
df[['score']].plot.line()

In [ ]:
name = 'lightgbm'
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_SHOTS_ON_TARGET_season_sum', 'TEAM_ATTACKS_season_sum', 'TEAM_ATTACKS_season_average', 'TEAM_GAME_LOST_season_sum', 'TEAM_ATTACKS_5_last_match_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_std', 'TEAM_CORNERS_5_last_match_std', 'TEAM_CORNERS_season_sum', 'TEAM_BALL_POSSESSION_5_last_match_average', 'TEAM_BALL_SAFE_season_std', 'TEAM_CORNERS_season_std', 'TEAM_FOULS_5_last_match_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_sum', 'TEAM_GOALS_5_last_match_std', 'TEAM_SAVES_5_last_match_std', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_PENALTIES_season_sum', 'TEAM_SHOTS_ON_TARGET_season_std', 'TEAM_BALL_SAFE_5_last_match_average', 'TEAM_PENALTIES_season_std', 'TEAM_SUBSTITUTIONS_season_average', 'TEAM_GAME_DRAW_season_sum', 'TEAM_YELLOWCARDS_season_sum', 'TEAM_SHOTS_OFF_TARGET_season_average', 'TEAM_SHOTS_TOTAL_season_std', 'TEAM_OFFSIDES_season_average', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_std', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_season_std', 'TEAM_FOULS_season_sum', 'TEAM_SUBSTITUTIONS_5_last_match_std', 'TEAM_SHOTS_OUTSIDEBOX_season_std', 'TEAM_ATTACKS_5_last_match_average', 'TEAM_REDCARDS_season_average', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_average', 'TEAM_INJURIES_season_std', 'TEAM_GOALS_5_last_match_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_sum', 'TEAM_INJURIES_season_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_average', 'TEAM_GAME_WON_5_last_match_average', 'TEAM_SUBSTITUTIONS_5_last_match_sum', 'TEAM_SUBSTITUTIONS_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_season_average', 'TEAM_GAME_DRAW_5_last_match_sum', 'TEAM_GAME_WON_5_last_match_sum', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_sum', 'TEAM_GAME_LOST_5_last_match_average', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_average', 'TEAM_PASSES_5_last_match_average', 'TEAM_PASSES_5_last_match_sum', 'TEAM_PASSES_season_sum', 'TEAM_SUCCESSFUL_PASSES_5_last_match_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_ATTACKS_season_sum', 'TEAM_GAME_LOST_season_sum', 'TEAM_CORNERS_season_sum', 'TEAM_BALL_SAFE_season_std', 'TEAM_CORNERS_season_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_sum', 'TEAM_SAVES_5_last_match_std', 'TEAM_PENALTIES_season_sum', 'TEAM_BALL_SAFE_5_last_match_average', 'TEAM_PENALTIES_season_std', 'TEAM_GAME_DRAW_season_sum', 'TEAM_SHOTS_OFF_TARGET_season_average', 'TEAM_OFFSIDES_season_average', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_std', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_std', 'TEAM_FOULS_season_sum', 'TEAM_SUBSTITUTIONS_5_last_match_std', 'TEAM_SHOTS_OUTSIDEBOX_season_std', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_average', 'TEAM_GOALS_5_last_match_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_season_average', 'TEAM_GAME_DRAW_5_last_match_sum', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_sum', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_average', 'TEAM_PASSES_5_last_match_sum', 'TEAM_PASSES_season_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_ATTACKS_season_sum', 'TEAM_CORNERS_season_sum', 'TEAM_BALL_SAFE_season_std', 'TEAM_CORNERS_season_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_sum', 'TEAM_PENALTIES_season_sum', 'TEAM_PENALTIES_season_std', 'TEAM_GAME_DRAW_season_sum', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_std', 'TEAM_FOULS_season_sum', 'TEAM_SUBSTITUTIONS_5_last_match_std', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_SHOTS_OFF_TARGET_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_5_last_match_std', 'TEAM_GAME_DRAW_5_last_match_sum', 'TEAM_PASSES_5_last_match_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_ATTACKS_season_sum', 'TEAM_BALL_SAFE_season_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_sum', 'TEAM_PENALTIES_season_std', 'TEAM_GAME_DRAW_season_sum', 'TEAM_FOULS_season_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_5_last_match_std']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_ATTACKS_season_sum', 'TEAM_BALL_SAFE_season_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_sum', 'TEAM_GAME_DRAW_season_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum']
df = test_features(name, features=features)
df[['score']].plot.line()

In [ ]:
name = 'xgb_classifier'
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_SHOTS_ON_TARGET_season_sum', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_ATTACKS_season_sum', 'TEAM_DANGEROUS_ATTACKS_season_sum', 'TEAM_CORNERS_season_average', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_SHOTS_ON_TARGET_5_last_match_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_std', 'TEAM_BALL_SAFE_season_average', 'TEAM_FOULS_5_last_match_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_sum', 'TEAM_GOALS_5_last_match_std', 'TEAM_CORNERS_5_last_match_sum', 'TEAM_SAVES_season_sum', 'TEAM_SHOTS_ON_TARGET_season_std', 'TEAM_OFFSIDES_season_std', 'TEAM_ATTACKS_season_std', 'TEAM_OFFSIDES_5_last_match_std', 'TEAM_GAME_DRAW_season_sum', 'TEAM_SUBSTITUTIONS_season_sum', 'TEAM_SHOTS_OFF_TARGET_season_average', 'TEAM_REDCARDS_season_std', 'TEAM_REDCARDS_season_sum', 'TEAM_SUCCESSFUL_PASSES_season_std', 'TEAM_FOULS_season_sum', 'TEAM_SHOTS_TOTAL_5_last_match_std', 'TEAM_SUBSTITUTIONS_5_last_match_std', 'TEAM_ATTACKS_5_last_match_sum', 'TEAM_ATTACKS_5_last_match_average', 'TEAM_YELLOWCARDS_5_last_match_sum', 'TEAM_REDCARDS_season_average', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_OFFSIDES_5_last_match_average', 'TEAM_SHOTS_INSIDEBOX_5_last_match_average', 'TEAM_GOALS_5_last_match_sum', 'TEAM_DANGEROUS_ATTACKS_5_last_match_average', 'TEAM_SHOTS_TOTAL_5_last_match_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_sum', 'TEAM_SHOTS_OFF_TARGET_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std', 'TEAM_GAME_WON_5_last_match_average', 'TEAM_FOULS_season_average', 'TEAM_PASSES_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_5_last_match_std', 'TEAM_SHOTS_OUTSIDEBOX_season_sum', 'TEAM_PENALTIES_5_last_match_std', 'TEAM_GAME_DRAW_5_last_match_sum', 'TEAM_INJURIES_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average', 'TEAM_SHOTS_TOTAL_5_last_match_sum', 'TEAM_PASSES_season_sum', 'TEAM_REDCARDS_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_5_last_match_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_ATTACKS_season_sum', 'TEAM_CORNERS_season_average', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_BALL_SAFE_season_average', 'TEAM_FOULS_5_last_match_std', 'TEAM_CORNERS_5_last_match_sum', 'TEAM_OFFSIDES_season_std', 'TEAM_OFFSIDES_5_last_match_std', 'TEAM_SHOTS_OFF_TARGET_season_average', 'TEAM_REDCARDS_season_std', 'TEAM_SHOTS_TOTAL_5_last_match_std', 'TEAM_SUBSTITUTIONS_5_last_match_std', 'TEAM_YELLOWCARDS_5_last_match_sum', 'TEAM_REDCARDS_season_average', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_DANGEROUS_ATTACKS_5_last_match_average', 'TEAM_SHOTS_TOTAL_5_last_match_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std', 'TEAM_FOULS_season_average', 'TEAM_PASSES_5_last_match_std', 'TEAM_SHOTS_OUTSIDEBOX_season_sum', 'TEAM_PENALTIES_5_last_match_std', 'TEAM_GAME_DRAW_5_last_match_sum', 'TEAM_INJURIES_5_last_match_average', 'TEAM_PASSES_season_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_ATTACKS_season_sum', 'TEAM_FOULS_5_last_match_std', 'TEAM_OFFSIDES_season_std', 'TEAM_OFFSIDES_5_last_match_std', 'TEAM_SHOTS_OFF_TARGET_season_average', 'TEAM_SHOTS_TOTAL_5_last_match_std', 'TEAM_SUBSTITUTIONS_5_last_match_std', 'TEAM_YELLOWCARDS_5_last_match_sum', 'TEAM_SHOTS_TOTAL_5_last_match_average', 'TEAM_FOULS_season_average', 'TEAM_GAME_DRAW_5_last_match_sum', 'TEAM_INJURIES_5_last_match_average']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_ATTACKS_season_sum', 'TEAM_OFFSIDES_5_last_match_std', 'TEAM_SHOTS_OFF_TARGET_season_average', 'TEAM_YELLOWCARDS_5_last_match_sum', 'TEAM_FOULS_season_average', 'TEAM_GAME_DRAW_5_last_match_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_ATTACKS_season_sum', 'TEAM_SHOTS_OFF_TARGET_season_average', 'TEAM_YELLOWCARDS_5_last_match_sum']
df = test_features(name, features=features)
df[['score']].plot.line()

In [ ]:
name = 'catboost'
features=['TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_ON_TARGET_5_last_match_std', 'TEAM_BALL_SAFE_season_average', 'TEAM_FOULS_5_last_match_std', 'TEAM_GOALS_5_last_match_std', 'TEAM_OFFSIDES_season_std', 'TEAM_ATTACKS_season_std', 'TEAM_GAME_DRAW_season_sum', 'TEAM_SUBSTITUTIONS_season_sum', 'TEAM_REDCARDS_season_std', 'TEAM_SUCCESSFUL_PASSES_season_std', 'TEAM_SHOTS_TOTAL_5_last_match_std', 'TEAM_SUBSTITUTIONS_5_last_match_std', 'TEAM_ATTACKS_5_last_match_average', 'TEAM_YELLOWCARDS_5_last_match_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_average', 'TEAM_SHOTS_TOTAL_5_last_match_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_sum', 'TEAM_SHOTS_OFF_TARGET_5_last_match_average', 'TEAM_GAME_WON_5_last_match_average', 'TEAM_PASSES_5_last_match_std', 'TEAM_PENALTIES_5_last_match_std', 'TEAM_INJURIES_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average', 'TEAM_PASSES_season_sum', 'TEAM_REDCARDS_5_last_match_std']
features=['TEAM_GAME_WON_season_sum', 'TEAM_ATTACKS_season_std', 'TEAM_GAME_DRAW_season_sum', 'TEAM_SUCCESSFUL_PASSES_season_std', 'TEAM_SUBSTITUTIONS_5_last_match_std', 'TEAM_ATTACKS_5_last_match_average', 'TEAM_YELLOWCARDS_5_last_match_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_sum', 'TEAM_GAME_WON_5_last_match_average', 'TEAM_INJURIES_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average']
features=['TEAM_GAME_WON_season_sum', 'TEAM_ATTACKS_season_std', 'TEAM_SUBSTITUTIONS_5_last_match_std', 'TEAM_ATTACKS_5_last_match_average', 'TEAM_SHOTS_INSIDEBOX_5_last_match_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_ATTACKS_season_std', 'TEAM_ATTACKS_5_last_match_average']
df = test_features(name, features=features)
df[['score']].plot.line()

In [ ]:
name = 'decision_tree'
features=['TEAM_GAME_WON_season_sum', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_ATTACKS_season_sum', 'TEAM_GOALS_season_sum', 'TEAM_SHOTS_TOTAL_season_average', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_GOALS_season_average', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_ATTACKS_5_last_match_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_std', 'TEAM_SUBSTITUTIONS_season_std', 'TEAM_BALL_POSSESSION_5_last_match_average', 'TEAM_BALL_SAFE_season_std', 'TEAM_BALL_SAFE_season_average', 'TEAM_FOULS_5_last_match_std', 'TEAM_YELLOWCARDS_season_std', 'TEAM_FOULS_season_std', 'TEAM_CORNERS_5_last_match_sum', 'TEAM_YELLOWCARDS_season_average', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_SAVES_season_sum', 'TEAM_OFFSIDES_season_std', 'TEAM_ATTACKS_season_std', 'TEAM_OFFSIDES_5_last_match_std', 'TEAM_GAME_DRAW_season_sum', 'TEAM_SHOTS_OFF_TARGET_season_average', 'TEAM_SHOTS_TOTAL_season_std', 'TEAM_CORNERS_5_last_match_average', 'TEAM_REDCARDS_season_sum', 'TEAM_OFFSIDES_season_average', 'TEAM_SHOTS_INSIDEBOX_5_last_match_std', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_season_std', 'TEAM_FOULS_season_sum', 'TEAM_SAVES_5_last_match_average', 'TEAM_ATTACKS_5_last_match_sum', 'TEAM_SHOTS_ON_TARGET_5_last_match_average', 'TEAM_SHOTS_OUTSIDEBOX_season_std', 'TEAM_INJURIES_season_sum', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_OFFSIDES_5_last_match_average', 'TEAM_FOULS_5_last_match_sum', 'TEAM_FOULS_5_last_match_average', 'TEAM_GOALS_5_last_match_sum', 'TEAM_GOALS_5_last_match_average', 'TEAM_SHOTS_ON_TARGET_5_last_match_sum', 'TEAM_SHOTS_TOTAL_5_last_match_average', 'TEAM_PASSES_season_std', 'TEAM_INJURIES_season_average', 'TEAM_SAVES_5_last_match_sum', 'TEAM_GAME_WON_5_last_match_average', 'TEAM_FOULS_season_average', 'TEAM_INJURIES_5_last_match_sum', 'TEAM_PENALTIES_5_last_match_std', 'TEAM_INJURIES_5_last_match_average', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_sum', 'TEAM_GAME_LOST_5_last_match_average', 'TEAM_PASSES_season_average', 'TEAM_PENALTIES_5_last_match_sum', 'TEAM_PASSES_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_5_last_match_average', 'TEAM_PENALTIES_5_last_match_average', 'TEAM_REDCARDS_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_5_last_match_sum', 'TEAM_REDCARDS_5_last_match_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_GOALS_season_sum', 'TEAM_SHOTS_TOTAL_season_average', 'TEAM_GOALS_season_average', 'TEAM_ATTACKS_5_last_match_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_std', 'TEAM_BALL_POSSESSION_5_last_match_average', 'TEAM_BALL_SAFE_season_average', 'TEAM_FOULS_5_last_match_std', 'TEAM_FOULS_season_std', 'TEAM_CORNERS_5_last_match_sum', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_ATTACKS_season_std', 'TEAM_OFFSIDES_5_last_match_std', 'TEAM_SHOTS_OFF_TARGET_season_average', 'TEAM_OFFSIDES_season_average', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_season_std', 'TEAM_FOULS_season_sum', 'TEAM_SHOTS_OUTSIDEBOX_season_std', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_OFFSIDES_5_last_match_average', 'TEAM_FOULS_5_last_match_average', 'TEAM_GOALS_5_last_match_sum', 'TEAM_GOALS_5_last_match_average', 'TEAM_PASSES_season_std', 'TEAM_SAVES_5_last_match_sum', 'TEAM_FOULS_season_average', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_sum', 'TEAM_PASSES_season_average', 'TEAM_PASSES_5_last_match_sum', 'TEAM_PENALTIES_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_5_last_match_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_ATTACKS_5_last_match_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_std', 'TEAM_FOULS_5_last_match_std', 'TEAM_CORNERS_5_last_match_sum', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_ATTACKS_season_std', 'TEAM_OFFSIDES_season_average', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_std', 'TEAM_SUCCESSFUL_PASSES_season_std', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_OFFSIDES_5_last_match_average', 'TEAM_GOALS_5_last_match_average', 'TEAM_PASSES_season_std', 'TEAM_SAVES_5_last_match_sum', 'TEAM_PASSES_5_last_match_sum', 'TEAM_PENALTIES_5_last_match_average']
features=['TEAM_GAME_WON_season_sum', 'TEAM_FOULS_5_last_match_std', 'TEAM_CORNERS_5_last_match_sum', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_ATTACKS_season_std', 'TEAM_SUCCESSFUL_PASSES_season_std', 'TEAM_GOALS_5_last_match_average', 'TEAM_PASSES_5_last_match_sum']
features=['TEAM_GAME_WON_season_sum', 'TEAM_BALL_SAFE_5_last_match_sum', 'TEAM_ATTACKS_season_std', 'TEAM_GOALS_5_last_match_average']
df = test_features(name, features=features)
df[['score']].plot.line()

In [ ]:
name = 'mlp'
features=['TEAM_GAME_WON_season_average', 'TEAM_GAME_LOST_season_average', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_DANGEROUS_ATTACKS_season_sum', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_SHOTS_ON_TARGET_season_sum', 'TEAM_CORNERS_season_average', 'TEAM_ATTACKS_5_last_match_std', 'TEAM_CORNERS_season_sum', 'TEAM_ATTACKS_season_std', 'TEAM_BALL_SAFE_season_sum', 'TEAM_GOALS_season_average', 'TEAM_GOALS_season_std', 'TEAM_BALL_SAFE_5_last_match_std', 'TEAM_YELLOWCARDS_season_std', 'TEAM_SHOTS_OFF_TARGET_season_std', 'TEAM_SAVES_5_last_match_std', 'TEAM_REDCARDS_season_std', 'TEAM_OFFSIDES_5_last_match_std', 'TEAM_FOULS_5_last_match_std', 'TEAM_YELLOWCARDS_5_last_match_average', 'TEAM_SAVES_season_sum', 'TEAM_GOALS_5_last_match_std', 'TEAM_PENALTIES_season_average', 'TEAM_YELLOWCARDS_season_sum', 'TEAM_GAME_DRAW_season_average', 'TEAM_SAVES_season_std', 'TEAM_SHOTS_INSIDEBOX_5_last_match_std', 'TEAM_SUBSTITUTIONS_season_sum', 'TEAM_BALL_SAFE_5_last_match_average', 'TEAM_OFFSIDES_season_std', 'TEAM_OFFSIDES_5_last_match_average', 'TEAM_PENALTIES_season_sum', 'TEAM_CORNERS_5_last_match_average', 'TEAM_SUBSTITUTIONS_season_average', 'TEAM_SHOTS_ON_TARGET_5_last_match_average', 'TEAM_FOULS_season_sum', 'TEAM_GAME_DRAW_season_sum', 'TEAM_SHOTS_ON_TARGET_season_std', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_std', 'TEAM_SHOTS_OFF_TARGET_season_sum', 'TEAM_ATTACKS_5_last_match_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_sum', 'TEAM_REDCARDS_season_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_average', 'TEAM_REDCARDS_season_sum', 'TEAM_GOALS_5_last_match_average', 'TEAM_SHOTS_INSIDEBOX_season_std', 'TEAM_INJURIES_5_last_match_std', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_PASSES_5_last_match_std', 'TEAM_INJURIES_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_season_average', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std', 'TEAM_PASSES_season_average', 'TEAM_GAME_WON_5_last_match_sum', 'TEAM_INJURIES_season_std', 'TEAM_SUCCESSFUL_PASSES_season_std', 'TEAM_PASSES_season_std', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average', 'TEAM_GAME_DRAW_5_last_match_sum', 'TEAM_INJURIES_5_last_match_sum', 'TEAM_SHOTS_TOTAL_5_last_match_sum', 'TEAM_GAME_DRAW_5_last_match_average', 'TEAM_GAME_WON_5_last_match_average', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_average', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_season_sum', 'TEAM_REDCARDS_5_last_match_std', 'TEAM_PASSES_5_last_match_sum', 'TEAM_PASSES_5_last_match_average', 'TEAM_PENALTIES_5_last_match_average', 'TEAM_REDCARDS_5_last_match_sum']
features=['TEAM_GAME_WON_season_average', 'TEAM_GAME_LOST_season_average', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_SHOTS_ON_TARGET_season_sum', 'TEAM_CORNERS_season_average', 'TEAM_CORNERS_season_sum', 'TEAM_GOALS_season_std', 'TEAM_REDCARDS_season_std', 'TEAM_SAVES_season_sum', 'TEAM_GOALS_5_last_match_std', 'TEAM_GAME_DRAW_season_average', 'TEAM_SUBSTITUTIONS_season_sum', 'TEAM_OFFSIDES_season_std', 'TEAM_CORNERS_5_last_match_average', 'TEAM_FOULS_season_sum', 'TEAM_GAME_DRAW_season_sum', 'TEAM_SHOTS_ON_TARGET_season_std', 'TEAM_SHOTS_OFF_TARGET_season_sum', 'TEAM_SHOTS_OFF_TARGET_5_last_match_sum', 'TEAM_SHOTS_OFF_TARGET_5_last_match_average', 'TEAM_GOALS_5_last_match_average', 'TEAM_INJURIES_5_last_match_std', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std', 'TEAM_GAME_WON_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_season_std', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average', 'TEAM_GAME_DRAW_5_last_match_sum', 'TEAM_INJURIES_5_last_match_sum', 'TEAM_SHOTS_TOTAL_5_last_match_sum', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_season_sum', 'TEAM_PASSES_5_last_match_sum', 'TEAM_PASSES_5_last_match_average']
features=['TEAM_GAME_WON_season_average', 'TEAM_GAME_LOST_season_average', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_CORNERS_season_average', 'TEAM_CORNERS_season_sum', 'TEAM_REDCARDS_season_std', 'TEAM_GAME_DRAW_season_average', 'TEAM_OFFSIDES_season_std', 'TEAM_FOULS_season_sum', 'TEAM_SHOTS_OFF_TARGET_season_sum', 'TEAM_GOALS_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average', 'TEAM_GAME_DRAW_5_last_match_sum', 'TEAM_SHOTS_TOTAL_5_last_match_sum', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_average', 'TEAM_PASSES_5_last_match_sum']
features=['TEAM_GAME_WON_season_average', 'TEAM_GAME_LOST_season_average', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_BALL_POSSESSION_season_average', 'TEAM_REDCARDS_season_std', 'TEAM_OFFSIDES_season_std', 'TEAM_GOALS_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average', 'TEAM_SHOTS_TOTAL_5_last_match_sum', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_average']
features=['TEAM_GAME_WON_season_average', 'TEAM_GAME_LOST_season_average', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_REDCARDS_season_std', 'TEAM_SHOTS_TOTAL_5_last_match_sum']
df = test_features(name, features=features)
df[['score']].plot.line()

In [ ]:
name = 'kneighbors'
features=['TEAM_GAME_WON_season_average', 'TEAM_GAME_WON_season_sum', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_SHOTS_ON_TARGET_season_sum', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_CORNERS_season_average', 'TEAM_YELLOWCARDS_5_last_match_std', 'TEAM_ATTACKS_season_std', 'TEAM_FOULS_season_std', 'TEAM_GOALS_season_average', 'TEAM_GOALS_season_std', 'TEAM_CORNERS_5_last_match_sum', 'TEAM_SUBSTITUTIONS_season_std', 'TEAM_YELLOWCARDS_season_std', 'TEAM_SAVES_5_last_match_std', 'TEAM_OFFSIDES_5_last_match_std', 'TEAM_BALL_SAFE_season_average', 'TEAM_BALL_POSSESSION_5_last_match_std', 'TEAM_OFFSIDES_5_last_match_sum', 'TEAM_SAVES_season_sum', 'TEAM_YELLOWCARDS_5_last_match_sum', 'TEAM_YELLOWCARDS_season_sum', 'TEAM_SAVES_season_std', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_std', 'TEAM_BALL_SAFE_5_last_match_average', 'TEAM_OFFSIDES_season_std', 'TEAM_SHOTS_TOTAL_5_last_match_std', 'TEAM_BALL_POSSESSION_5_last_match_average', 'TEAM_CORNERS_5_last_match_average', 'TEAM_SUBSTITUTIONS_season_average', 'TEAM_OFFSIDES_season_sum', 'TEAM_SHOTS_ON_TARGET_season_std', 'TEAM_SHOTS_ON_TARGET_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_std', 'TEAM_SHOTS_TOTAL_season_std', 'TEAM_ATTACKS_5_last_match_average', 'TEAM_ATTACKS_5_last_match_sum', 'TEAM_REDCARDS_season_average', 'TEAM_SHOTS_OFF_TARGET_5_last_match_average', 'TEAM_DANGEROUS_ATTACKS_5_last_match_sum', 'TEAM_SHOTS_INSIDEBOX_season_std', 'TEAM_FOULS_season_average', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_GOALS_5_last_match_sum', 'TEAM_SUBSTITUTIONS_5_last_match_average', 'TEAM_SUBSTITUTIONS_5_last_match_sum', 'TEAM_GAME_LOST_5_last_match_average', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std', 'TEAM_SUCCESSFUL_PASSES_5_last_match_std', 'TEAM_GAME_WON_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_season_std', 'TEAM_GAME_LOST_5_last_match_sum', 'TEAM_PENALTIES_5_last_match_std', 'TEAM_SHOTS_TOTAL_5_last_match_average', 'TEAM_SHOTS_TOTAL_5_last_match_sum', 'TEAM_SHOTS_OUTSIDEBOX_season_average', 'TEAM_GAME_DRAW_5_last_match_average', 'TEAM_GAME_WON_5_last_match_average', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_sum', 'TEAM_PENALTIES_5_last_match_sum', 'TEAM_REDCARDS_5_last_match_std', 'TEAM_PASSES_5_last_match_average', 'TEAM_PENALTIES_5_last_match_average']
features=['TEAM_GAME_WON_season_average', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_CORNERS_season_average', 'TEAM_ATTACKS_season_std', 'TEAM_FOULS_season_std', 'TEAM_GOALS_season_std', 'TEAM_SUBSTITUTIONS_season_std', 'TEAM_SAVES_5_last_match_std', 'TEAM_OFFSIDES_5_last_match_sum', 'TEAM_SAVES_season_sum', 'TEAM_YELLOWCARDS_season_sum', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_std', 'TEAM_SHOTS_TOTAL_5_last_match_std', 'TEAM_BALL_POSSESSION_5_last_match_average', 'TEAM_CORNERS_5_last_match_average', 'TEAM_SUBSTITUTIONS_season_average', 'TEAM_SHOTS_ON_TARGET_season_std', 'TEAM_SHOTS_TOTAL_season_std', 'TEAM_ATTACKS_5_last_match_sum', 'TEAM_SHOTS_INSIDEBOX_season_std', 'TEAM_SHOTS_INSIDEBOX_5_last_match_sum', 'TEAM_GOALS_5_last_match_sum', 'TEAM_SUBSTITUTIONS_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std', 'TEAM_SUCCESSFUL_PASSES_5_last_match_std', 'TEAM_GAME_WON_5_last_match_sum', 'TEAM_GAME_LOST_5_last_match_sum', 'TEAM_SHOTS_TOTAL_5_last_match_sum', 'TEAM_SHOTS_OUTSIDEBOX_season_average', 'TEAM_GAME_WON_5_last_match_average', 'TEAM_PASSES_5_last_match_average']
features=['TEAM_GAME_WON_season_average', 'TEAM_CORNERS_season_average', 'TEAM_ATTACKS_season_std', 'TEAM_GOALS_season_std', 'TEAM_SUBSTITUTIONS_season_std', 'TEAM_OFFSIDES_5_last_match_sum', 'TEAM_YELLOWCARDS_season_sum', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_std', 'TEAM_BALL_POSSESSION_5_last_match_average', 'TEAM_SHOTS_ON_TARGET_season_std', 'TEAM_SHOTS_TOTAL_season_std', 'TEAM_SHOTS_INSIDEBOX_season_std', 'TEAM_GOALS_5_last_match_sum', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std', 'TEAM_GAME_LOST_5_last_match_sum', 'TEAM_SHOTS_OUTSIDEBOX_season_average']
features=['TEAM_GAME_WON_season_average', 'TEAM_ATTACKS_season_std', 'TEAM_SUBSTITUTIONS_season_std', 'TEAM_OFFSIDES_5_last_match_sum', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_std', 'TEAM_BALL_POSSESSION_5_last_match_average', 'TEAM_GOALS_5_last_match_sum']
features=['TEAM_GAME_WON_season_average', 'TEAM_OFFSIDES_5_last_match_sum', 'TEAM_SHOTS_OUTSIDEBOX_5_last_match_std', 'TEAM_BALL_POSSESSION_5_last_match_average']
df = test_features(name, features=features)
df[['score']].plot.line()

In [ ]:
def plot_features(folder = '../data/output/features/'):
   import os, pandas
   df = None
   for filename in os.listdir(folder):
      # if not os.path.isfile(filename): continue
      model = filename.split('.')[0]
      df_tmp = pandas.read_csv(f'{folder}{filename}')
      df_tmp[model] = df_tmp['score']
      if df is None:
         df = df_tmp[[model]]
      else:
         df[model] = df_tmp[[model]]
   df.plot.line()

In [ ]:
plot_features('../data/output/features/')

In [ ]:
plot_features('../data/output/all_features/')